In [ ]:
%matplotlib inline
import mne
import matplotlib.pyplot as plt

fname = "oddball-epo.fif"

epochs = mne.read_epochs(fname)

## Evokeds

Finally, if we average an epoched dataset over trials, we can use the `mne.Evoked` object.

In [ ]:
target = epochs["target"].average()
target

In [ ]:
standard = epochs["standard"].average()

To quickly investigate evoked activity, the `Evoked` object has a number of plotting functions available.

In [ ]:
target.plot_joint();

For condition contrasts, you can use `mne.combine.evoked`:

In [ ]:
diff = mne.combine_evoked((target, -standard), weights='equal')
diff.plot_joint(times=.35);

Or as an image:

In [ ]:
diff.plot_image();

Because we have a 10/20 electrode layout, we can easily use a somewhat nicer layout:

In [ ]:
rois = mne.channels.make_1020_channel_selections(diff.info, midline="z12")
diff.plot_image(group_by=rois, show=False, show_names="all");

To contrast multiple conditions, `mne.viz.plot_compare_evokeds` is available:

In [ ]:
mne.viz.plot_compare_evokeds({"standard": standard,
                              "target": target}, picks=[13]);

## Time-Frequency stuff

For an overview over the spectral shape of the data, we can use a plotting method of `raw`, `raw.plot_psd`:

In [ ]:
epochs_for_tfr = mne.read_epochs("oddball-long-epo.fif")

In [ ]:
epochs_for_tfr.plot_psd(fmin=2, fmax=20);

But what about the time/frequency correlates of the Oddball effect?

We will extract power per time and frequency with Morlet wavelets.

In [ ]:
from mne.time_frequency import tfr_morlet

In [ ]:
freqs = list(range(3, 30))
tfr_target = tfr_morlet(epochs_for_tfr["target"], freqs, 3, return_itc=False)
tfr_standard = tfr_morlet(epochs_for_tfr["standard"], freqs, 3, return_itc=False)

Time-frequency data (single trial or averaged) is stored in TFR objects. These objects behave in many ways like Evoked objects ...

In [ ]:
tfr_contrast = mne.combine_evoked((tfr_standard, tfr_target), (-.5, .5))
tfr_contrast.apply_baseline((None, 0))

Plotting time-frequencyy activity (event-related spectral perturbations): observe the alpha-band ERD and the time-frequency correlates of the P3 effect.

In [ ]:
tfr_contrast.plot_joint();

In [ ]:
tfr_contrast.plot(picks=[27]);

In [ ]:
del epochs_for_tfr

## Statistics

Remember what the data look like:

In [ ]:
diff.plot_image(group_by=rois, show=False, show_names="all");

Can we statistically threshold this image to see which effects are reliable?

### Cluster-based permutation stats

Exploratory analysis with nonparametric control of the error rate is commonly done with
cluster-based permutation tests (i.e., Maris 2012). To cluster across space, we first need a
channel adjacency matrix.

In [ ]:
from mne.channels import find_ch_connectivity
connectivity, ch_names = find_ch_connectivity(epochs.info, ch_type='eeg')
plt.imshow(connectivity.toarray(), cmap="Greys")

Now we need the data in the right shape. Sadly, because the space dimension needs
to be last, we need to manually swap the time and space axes.

In [ ]:
epochs.pick_types(eeg=True)
target_epochs, standard_epochs = epochs["target"].get_data(), epochs["standard"].get_data()
target_epochs.shape, standard_epochs.shape

In [ ]:
target_epochs = target_epochs.swapaxes(1, 2)
standard_epochs = standard_epochs.swapaxes(1, 2)
target_epochs.shape, standard_epochs.shape

MNE has various cluster-based permutation test options. Here, we test for single-trial
differences between conditions with `mne.stats.spatio_temporal_cluster_test`.

We use threshold-free cluster enhancement to reduce the number of parameters.

Warning: the next cell takes a lot of time and computational power.

In [ ]:
from mne.stats import spatio_temporal_cluster_test

mne.set_log_level(True)
tfce = dict(start=.2, step=.5)  # decrease both for real analyses
cluster_stats = spatio_temporal_cluster_test([target_epochs, standard_epochs],
                                             threshold=tfce,
                                             n_permutations=200,  # way too low, increase for real analyses
                                             n_jobs=1,  # increase for decent CPUs
                                             connectivity=connectivity)
T_obs, clusters, p_values, _ = cluster_stats

Now we can visualise the *t* values over time and space ...

In [ ]:
extent = (*epochs.times[[0, -1]], 0, len(epochs.ch_names))
im = plt.imshow(T_obs.T, aspect="auto", cmap="RdBu_r",
                vmin=-100, vmax=100, extent=extent
          )
plt.colorbar(im)

... and the p-values.

In [ ]:
plt.hist(p_values)

alpha = .01
print(sum(p_values < alpha))

We can use the resulting mask to mask the image:

In [ ]:
pvals = p_values.reshape(T_obs.shape).T < alpha

diff.plot_image(group_by=rois, show=False, show_names="all", mask=pvals);

### Parametric stats
Sometimes, e.g. because we wish to test a specific hypothesis, cluster-based permutation tests are too much.
We can also simply access the data in array form and test with parametric (or nonparametric) tests.

For this, we first need to identify the spatial and temporal coordinates of an effect we want to test -
for example, the N2 at Cz.

In [ ]:
time_mask = (.2 < epochs.times) & (epochs.times < .25)
electrode_pz = epochs.ch_names.index("Cz")
plt.plot(time_mask)

Now we extract the target data. Reminder: the shape of epochs data is (trial, channel, time)

In [ ]:
epochs["target"].get_data().shape

In [ ]:
cond_a = epochs["target"].get_data()[:, electrode_pz, time_mask].mean(-1)
cond_b = epochs["standard"].get_data()[:, electrode_pz, time_mask].mean(-1)

In [ ]:
cond_a.shape

Now we can simply use ordinary tests on these statistics.

In [ ]:
from scipy.stats import ttest_ind, wilcoxon

In [ ]:
ttest_ind(cond_a, cond_b)

In [ ]:
wilcoxon(cond_a, cond_b)

It is also straight-forward to convert the data into a (pandas) dataframe.

In [ ]:
df = epochs.to_data_frame()
df.head(20)

In [ ]:
df_cz = df.query("200 < time < 250")["Cz"].groupby(["epoch", "condition"]).mean().reset_index()
df_cz.head()

In [ ]:
import seaborn as sns
sns.factorplot(y="Cz", data=df_cz, x="condition")